## Chat engine in Condense Question mode with explicitely specified vector_retriever and response_synthesizer
### trying to integrate marvins ai_model

In [1]:
from marvin import ai_model
from llama_index.bridge.pydantic import BaseModel, Field
from llama_index.callbacks import CallbackManager, TokenCountingHandler
from llama_index.llms import OpenAI
from llama_index.llms import ChatMessage, MessageRole
from llama_index import (
    VectorStoreIndex, 
    SimpleDirectoryReader,
    ServiceContext,
    set_global_service_context,
    get_response_synthesizer,
)
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
)
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser.extractors.marvin_metadata_extractor import (
    MarvinMetadataExtractor,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

from openai import log as openai_log
import tiktoken

import logging
import sys

from dotenv import load_dotenv
import os
import certifi

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai_log = "debug"

load_dotenv()
# load_dotenv("../.env")
# workaround for mac to solve "SSL: CERTIFICATE_VERIFY_FAILED Error"


os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["SSL_CERT_FILE"] = certifi.where()

API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_MODEL = "gpt-3.5-turbo"

In [2]:
CATEGORY_LABELS = [
    "Technical",
    "Science-Fiction",
    "Poetry",
    "Fantasy",
    "Mystery",
    "Romance",
    "Historical",
    "Fiction",
    "Self-Help",
    "Biography",
    "Travelogue",
    "Horror",
    "Comedy",
    "Thriller",
    "Science",
    "Philosophy",
    "Memoir",
    "Cookbook",
    "Business",
    "Drama",
    "Satire",
]

@ai_model
class QueryDocument(BaseModel):
    #name: str = Field(..., description="The name of the document")
    description: str = Field(..., description="a brief summary of the document content.")
    text_category: str = Field(...,description=f"best matching text category from the following list: {str(CATEGORY_LABELS)}")
    
    # def __init__(self, name):
    #     super().__init__()
    #     self.name = name



In [3]:

documents = SimpleDirectoryReader(
    input_files=["./data/test2.txt"],
    encoding="utf-8"
).load_data()

llm = OpenAI(model=OPENAI_MODEL, temperature=0, max_tokens=512)

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model(OPENAI_MODEL).encode
)
callback_manager = CallbackManager([token_counter])

#CHAT_MODE = "technical"


In [4]:


metadata_extractor = MetadataExtractor(
    extractors=[
        MarvinMetadataExtractor(
            marvin_model=QueryDocument, 
            llm_model_string=OPENAI_MODEL,
            show_progress = True,
            callback_manager=callback_manager,

        ),
    ],
)

logging.info(f"Number of used tokens: {token_counter.total_embedding_token_count}")

text_splitter = TokenTextSplitter(
    separator=" ", 
    chunk_size=1024, 
    chunk_overlap=128,
    callback_manager=callback_manager
)


INFO:root:Number of used tokens: 0
Number of used tokens: 0


In [5]:


node_parser = SimpleNodeParser(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
    callback_manager = callback_manager,
)

logging.info(f"Number of used tokens: {token_counter.total_embedding_token_count}")

INFO:root:Number of used tokens: 0
Number of used tokens: 0


In [6]:

nodes = node_parser.get_nodes_from_documents(documents)

logging.info(f"Number of used tokens: {token_counter.total_embedding_token_count}")

from pprint import pprint

for node in nodes:
    pprint(node.metadata)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Albert Einstein (* 14. März 1879 in Ulm; † 18. ...
> Adding chunk: Albert Einstein (* 14. März 1879 in Ulm; † 18. ...
DEBUG:asyncio:Using selector: KqueueSelector
Using selector: KqueueSelector
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "The user will provide context as text that you need to parse into a\\nstructured form. To validate your response, you must call the\\n`FormatResponse` function. Use the provided text to extract or infer\\nany parameters needed by `FormatResponse`, including any missing\\ndata.\\n\\nIt is Tuesday, 26 September 2023 at 07:47:22 PM UTC."}, {"role": "user", "content": "The text to parse: Albert Einstein (* 14. M\\u00e4rz 1879 in Ulm; \\u2020 18.

In [7]:
text_category = nodes[0].metadata["marvin_metadata"].get("text_category")

In [8]:
system_prompt = f"""You are a chatbot that responds to all questions about the content of a given document, which is available in the form of embeddings in the given vector database. The user gives you instructions on which questions to answer. 
    When you write the answers, you need to make sure that the user's expectations are met. Remember that you are an accurate and experienced writer 
    and you write unique and short answers in the style of a {text_category} text. Don't add anything hallucinatory.
    Use friendly, easy-to-read language, and if it is a technical or scientific text, please stay correct and focused.
    Responses should be no longer than 10 sentences, unless the user explicitly specifies the number of sentences.
"""
service_context = ServiceContext.from_defaults(
    llm=llm, 
    chunk_size=1024, 
    chunk_overlap=152,
    #system_prompt=system_prompt,
    callback_manager=callback_manager,
)
set_global_service_context(service_context)


In [9]:
vector_index = VectorStoreIndex(nodes, service_context=service_context) # openai api is called with whole text to make the embeddings
logging.info(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["[Excerpt from document] marvin_metadata: {\'description\': \'Albert Einstein was a Swiss-American theoretical physicist of German origin. He was born on March 14, 1879 in Ulm and died on April 18, 1955 in Princeton, New Jersey. Einstein had Swiss citizenship from 1901 and additionally obtained US citizenship in 1940. He was also a German citizen from 1914 to 1934. Einstein is considered one of the most significant physicists in the history of science and one of the most well-known scientists of modern times. His research on the structure of matter, space and time, as well as the nature of gravitation, significantly changed the previously accepted Newtonian worldview. In a survey conducted by the journal Physics World in 1999, Albert Einstein was voted the m

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/logging/__init__.py", line 1098, in emit
    msg = self.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/logging/__init__.py", line 942, in format
    return fmt.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
 

In [10]:
vector_retriever = VectorIndexRetriever(
    index=vector_index,
    similarity_top=2,

)

vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=get_response_synthesizer(),
    callback_manager=callback_manager,
)
logging.info(f"Number of used tokens: {token_counter.total_embedding_token_count}")

INFO:root:Number of used tokens: 450
Number of used tokens: 450


In [11]:
from llama_index.chat_engine.condense_question import CondenseQuestionChatEngine

# list of `ChatMessage` objects
# custom_chat_history = [
#     ChatMessage(
#         role=MessageRole.USER, 
#         content='Hello assistant, we are having a insightful discussion about the given text content.'
#     ), 
#     ChatMessage(
#         role=MessageRole.ASSISTANT, 
#         content='Okay, sounds good.'
#     )
# ]
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=vector_query_engine, 
    #condense_question_prompt=custom_prompt,
    #chat_history=custom_chat_history,
    
    verbose=True,
    callback_manager=callback_manager,
    
)

In [13]:
response = chat_engine.chat("What did Einstein do?")
logging.info(f"Number of used tokens: {token_counter.total_embedding_token_count}")

DEBUG:llama_index.chat_engine.condense_question:user: What did Einstein do?
assistant: Einstein's contribution or achievement was his groundbreaking research on the structure of matter, space and time, as well as the nature of gravitation. His work significantly changed the previously accepted Newtonian worldview and had a profound impact on the field of physics. He is considered one of the most significant physicists in the history of science and is widely recognized as one of the most well-known scientists of modern times.
user: What did Einstein do?
assistant: Einstein's contribution or achievement was his groundbreaking research on the structure of matter, space and time, as well as the nature of gravitation. His work significantly changed the previously accepted Newtonian worldview and had a profound impact on the field of physics. He is considered one of the most significant physicists in the history of science and is widely recognized as one of the most well-known scientists of 

In [15]:
response.response

'Einstein made significant contributions and achievements in the field of physics. His research on the structure of matter, space and time, as well as the nature of gravitation, significantly changed the previously accepted Newtonian worldview. He is considered one of the most significant physicists in the history of science and one of the most well-known scientists of modern times. In a survey conducted by the journal Physics World in 1999, Albert Einstein was voted the most significant physicist of all time, ahead of Isaac Newton, James Clerk Maxwell, Niels Bohr, and Werner Heisenberg.'

In [16]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='83a538bb-4d34-4815-938f-f3aaab4c105d', embedding=None, metadata={'marvin_metadata': {'description': 'Albert Einstein was a Swiss-American theoretical physicist of German origin. He was born on March 14, 1879 in Ulm and died on April 18, 1955 in Princeton, New Jersey. Einstein had Swiss citizenship from 1901 and additionally obtained US citizenship in 1940. He was also a German citizen from 1914 to 1934. Einstein is considered one of the most significant physicists in the history of science and one of the most well-known scientists of modern times. His research on the structure of matter, space and time, as well as the nature of gravitation, significantly changed the previously accepted Newtonian worldview. In a survey conducted by the journal Physics World in 1999, Albert Einstein was voted the most significant physicist of all time, ahead of Isaac Newton, James Clerk Maxwell, Niels Bohr, and Werner Heisenberg.', 'text_category': 'Biography'}}, exclude

In [17]:
response = chat_engine.chat("What was Einsteins' favorite food?")
logging.info(f"Number of used tokens: {token_counter.total_embedding_token_count}")

DEBUG:llama_index.chat_engine.condense_question:user: What did Einstein do?
assistant: Einstein's contribution or achievement was his groundbreaking research on the structure of matter, space and time, as well as the nature of gravitation. His work significantly changed the previously accepted Newtonian worldview and had a profound impact on the field of physics. He is considered one of the most significant physicists in the history of science and is widely recognized as one of the most well-known scientists of modern times.
user: What did Einstein do?
assistant: Einstein made significant contributions and achievements in the field of physics. His research on the structure of matter, space and time, as well as the nature of gravitation, significantly changed the previously accepted Newtonian worldview. He is considered one of the most significant physicists in the history of science and one of the most well-known scientists of modern times. In a survey conducted by the journal Physics 

In [18]:
response.response

"I'm sorry, but the given context does not provide any information about Albert Einstein's favorite food."

- https://github.com/jerryjliu/llama_index/blob/main/docs/examples/index_structs/doc_summary/DocSummary.ipynb
- https://betterprogramming.pub/llamaindex-0-6-0-a-new-query-interface-over-your-data-331996d47e89
- https://gpt-index.readthedocs.io/en/latest/examples/query_engine/CustomRetrievers.html
- https://gpt-index.readthedocs.io/en/latest/core_modules/query_modules/chat_engines/usage_pattern.html

- https://gpt-index.readthedocs.io/en/latest/examples/metadata_extraction/MarvinMetadataExtractorDemo.html